# Imports

In [1]:
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
# this adds the src folder in the sys path, where the metric_utils.py file is
# not needed if this notebook is in the same folder, but uncomment to access from the data subfolders
sys.path.append( '..' )
from metric_utils import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Result path

In [60]:
dataset_name = 'jigsaw'
model_name = 'bert'
run_name = "run 1"

# change to f'../../results/{dataset_name}' when using inside one of the data subfolders
run_folder = f'../../results/{dataset_name}/{run_name}'
test_csv_filepath = os.path.join(run_folder, 'test.csv')

model_folder = os.path.join(run_folder, model_name) # for this particular model
normal_folder = os.path.join(model_folder, 'normal')
dp_folder = os.path.join(model_folder, 'epsilon 3.0')

result_filepath = os.path.join(normal_folder, 'results.csv')
dp_result_filepath = os.path.join(dp_folder, 'results.csv')

# Dataset
The dataset can be preprocessed from the original dataset to be used here. To simplify things, I saved the preprocessed datasets during the tokenizing process as csv files and then downloaded them in the corresponding dataset folder of [`result`](../results/) directory.

You can recreate the processed datasets using the tokenize notebooks for that particular dataset. That would give you train, test and validation csv files as well as the tokenized data.

In [47]:
result = pd.read_csv(result_filepath)
dp_result = pd.read_csv(dp_result_filepath)

test_df = pd.read_csv(test_csv_filepath)
# df.drop(columns=['comment_text', 'labels'], inplace=True)
# df.to_csv(test_csv_filepath, index=False)
test_df.head(2)

,id,labels,male,female,transgender,white,black,asian
0,7084460,1,NaN,NaN,NaN,NaN,NaN,NaN
1,7141509,1,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
# Doing this reduces the test file size, makes things easier with git
if 'comment_text' in test_df.columns:
    test_df.drop(columns='comment_text', inplace=True)
    test_df.to_csv(test_csv_filepath, index=False)

test_df.fillna(0, inplace=True)

## Split test result

In [49]:
# only calculate test result
result = result[result['split']=='test']
dp_result = dp_result[dp_result['split']=='test']

# drop split column
result.drop(columns=['split'], inplace=True)
dp_result.drop(columns=['split'], inplace=True)

## Merge

In [50]:
extra_columns = [col for col in df.columns if col in result.columns and col!=id_column]

result = result.merge(df.drop(columns=extra_columns), on=id_column, how='inner').reset_index(drop=True)
dp_result = dp_result.merge(df.drop(columns=extra_columns), on=id_column, how='inner').reset_index(drop=True)

# Evaluation

## Convert probability to prediction

In [8]:
result[prediction_column] = result[probability_column] >=0.5
dp_result[prediction_column] = dp_result[probability_column] >=0.5

## Identity groups

In [9]:
group_map = {
    'gender': {
        'unprivileged':['female'],
        'privileged':['male']
    },
    'race': {
        'unprivileged':['black'],
        'privileged': ['white']
    }
}

identities = []
for group_key in group_map.keys():
    subgroup_map = group_map[group_key]
    for subgroup_key in subgroup_map.keys():
        identities.extend(subgroup_map[subgroup_key])

print(identities)

['female', 'male', 'black', 'white']


### Binarize identity and target columns

In [10]:
result = binarize(result, [target_column] + identities)
dp_result = binarize(dp_result, [target_column] + identities)

## Bias

In [11]:
bias_results = {
   # make sure your calculate bias method returns bias metrics in the same order
   'fairness_metrics': ['demographic parity', 'Equality of Opportunity (w.r.t y = 1)',
'Equality of Opportunity (w.r.t y = 0)', 'Equality of Odds', 'unprotected-accuracy',
'protected-accuracy', 'accuracy']
}

for group_key in group_map.keys():
   subgroup_map = group_map[group_key]
   privileged_group = subgroup_map['privileged']
   unprivileged_group = subgroup_map['unprivileged']

   bias_results[group_key] = calculate_bias(result, privileged_group, unprivileged_group)
   bias_results[group_key+'_DP'] = calculate_bias(dp_result, privileged_group, unprivileged_group)

bias_results = pd.DataFrame(bias_results) 
bias_results

,fairness_metrics,gender,gender_DP,race,race_DP
0,demographic parity,0.998580,0.975852,0.898132,0.797963
1,Equality of Opportunity (w.r.t y = 1),0.946161,0.978125,0.916765,0.903395
2,Equality of Opportunity (w.r.t y = 0),0.970549,0.980068,0.962048,0.938017
3,Equality of Odds,0.958355,0.979097,0.939406,0.920706
4,unprotected-accuracy,0.868178,0.822829,0.721419,0.712221
5,protected-accuracy,0.847538,0.806818,0.720713,0.699491
6,accuracy,0.857858,0.814823,0.721066,0.705856


In [12]:
bias_results.round(3).to_csv(os.path.join(dp_folder, 'bias.csv'), index=False)

## Overall metrics

In [13]:
overall_results = {
    'metrics': ['size', 'auc', 'accuracy', 'f1_score', 
    'precision', 'recall', 'false positive rate',
    'bnsp_auc', 'bpsn_auc']
}

for group_key in group_map.keys():
    subgroup_map = group_map[group_key]
    privileged_group = subgroup_map['privileged']
    unprivileged_group = subgroup_map['unprivileged']

    privileged_group_name = ','.join(privileged_group)
    unprivileged_group_name = ','.join(unprivileged_group)

    overall_results[privileged_group_name] = calculate_metrics(result, privileged_group)
    overall_results[privileged_group_name + '_DP'] = calculate_metrics(dp_result, privileged_group)

    overall_results[unprivileged_group_name] = calculate_metrics(result, unprivileged_group)
    overall_results[unprivileged_group_name + '_DP'] = calculate_metrics(dp_result, unprivileged_group)

overall_results['Total'] = calculate_metrics(result, [])
overall_results['Total_DP'] = calculate_metrics(dp_result, [])

overall_results = pd.DataFrame(overall_results) 
overall_results

,metrics,male,male_DP,female,female_DP,white,white_DP,black,black_DP,Total,Total_DP
0,size,2112.000000,2112.000000,2602.000000,2602.000000,1178.000000,1178.000000,761.000000,761.000000,97320.000000,97320.000000
1,auc,0.897062,0.849390,0.898539,0.850251,0.792065,0.771067,0.808122,0.737525,0.941883,0.894798
2,accuracy,0.847538,0.806818,0.868178,0.822829,0.720713,0.699491,0.721419,0.712221,0.925390,0.909094
3,f1_score,0.591371,0.531034,0.579141,0.515247,0.602177,0.602247,0.646667,0.598165,0.614494,0.553024
4,precision,0.497863,0.420000,0.507527,0.407654,0.525316,0.499069,0.548023,0.545151,0.523332,0.455476
5,recall,0.728125,0.721875,0.674286,0.700000,0.705382,0.759207,0.788618,0.662602,0.744117,0.703742
6,false positive rate,0.131138,0.178013,0.101687,0.158082,0.272727,0.326061,0.310680,0.264078,0.058866,0.073071
7,bnsp_auc,0.951126,0.927555,0.942339,0.920170,0.947406,0.918442,0.955836,0.879579,NaN,NaN
8,bpsn_auc,0.888014,0.813385,0.902654,0.824310,0.812069,0.751548,0.792129,0.779186,NaN,NaN


In [ ]:
overall_results.round(3).to_csv(os.path.join(dp_folder, 'overall_results.csv'), index=False)

## Overall bias auc

In [ ]:
overall_auc = overall_results[overall_results['metrics']=='auc']['Total'].values[0]

In [24]:
temp = overall_results.set_index('metrics').T
privileged, unprivileged = [], []
for group_key in group_map.keys():
    privileged.extend(group_map[group_key]['privileged'])
    unprivileged.extend(group_map[group_key]['unprivileged'])

privileged_bias_auc = get_final_metric(temp[temp.index.isin(privileged)], overall_auc)
unprivileged_bias_auc = get_final_metric(temp[temp.index.isin(unprivileged)], overall_auc)

privileged_bias_auc_dp = get_final_metric(
    temp[temp.index.isin([i+'_DP' for i in privileged])], 
    overall_auc
)

unprivileged_bias_auc_dp = get_final_metric(
    temp[temp.index.isin([i+'_DP' for i in unprivileged])], 
    overall_auc
)
print(privileged_bias_auc, unprivileged_bias_auc, privileged_bias_auc_dp, unprivileged_bias_auc_dp)

0.8927666570365307 0.8934576301082992 0.8620616692527434 0.8555933902575662


# Count
The the `train.csv` file from `experiment/run` folders for this corresponding run. And manually copy it in the path run_folder points to.

In [28]:
train = pd.read_csv(os.path.join(run_folder, 'train.csv'))

In [57]:
count_dict = {
    'Identity':identities,
    '0 (train)':[],
    '1 (train)':[],
    '0 (test)':[],
    '1 (test)':[],
}
for identity in identities:
    train_neg, train_pos = train[train[identity]>=0.5][target_column].value_counts().to_numpy()
    test_neg, test_pos = df[df[identity]>=0.5][target_column].value_counts().to_numpy()
    count_dict['0 (train)'].append(train_neg)
    count_dict['1 (train)'].append(train_pos)
    count_dict['0 (test)'].append(test_neg)
    count_dict['1 (test)'].append(test_pos)

count_df = pd.DataFrame(count_dict)
print(count_df)

  Identity  0 (train)  1 (train)  0 (test)  1 (test)
0   female      36869       5855      4463       692
1     male      30123       5386      3716       670
2    black       8260       3748      1016       503
3    white      14473       5683      1710       742


In [62]:
count_df.to_csv(os.path.join(run_folder, 'count.csv'), index=False)